In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range = 30,
    zoom_range = 0.15,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.15,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

valid_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

In [3]:
train_generator = train_datagen.flow_from_directory(
  'dogs/train/',
  target_size = (224,224),
  color_mode = 'rgb',
  batch_size = 10,
  class_mode = 'binary',
  shuffle = True,
  seed = 42
)

valid_generator = valid_datagen.flow_from_directory(
  'dogs/valid/',
  target_size = (224,224),
  color_mode = 'rgb',
  batch_size = 10,
  class_mode = 'binary',
  shuffle = False,
  seed = 42
)

test_generator = test_datagen.flow_from_directory(
  'dogs/test/',
  target_size = (224,224),
  color_mode = 'rgb',
  batch_size = 10,
  class_mode = 'binary',
  shuffle = False,
  seed = 42
)

Found 120 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [4]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(Conv2D(filters=32, input_shape=(224, 224, 3),
  kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
  optimizer='adam', metrics=['accuracy'])

step_size_train = train_generator.n // train_generator.batch_size
step_size_valid = valid_generator.n // valid_generator.batch_size
step_size_test = test_generator.n // test_generator.batch_size

callbacks = [EarlyStopping(monitor='val_loss', patience=5,
  restore_best_weights=True)]

history = model.fit(train_generator, steps_per_epoch = step_size_train,
  epochs = 50, callbacks = callbacks, validation_data = valid_generator,
  validation_steps = step_size_valid, verbose = 1)

Epoch 1/50
12/12 [==============================] - 3s 135ms/step - loss: 542.7184 - accuracy: 0.4505 - val_loss: 23.6520 - val_accuracy: 0.5000
Epoch 2/50
12/12 [==============================] - 1s 81ms/step - loss: 7.0623 - accuracy: 0.5435 - val_loss: 1.2436 - val_accuracy: 0.5000
Epoch 3/50
12/12 [==============================] - 1s 82ms/step - loss: 0.6827 - accuracy: 0.5701 - val_loss: 1.4234 - val_accuracy: 0.5500
Epoch 4/50
12/12 [==============================] - 1s 82ms/step - loss: 0.6517 - accuracy: 0.5460 - val_loss: 1.2014 - val_accuracy: 0.5000
Epoch 5/50
12/12 [==============================] - 1s 81ms/step - loss: 0.6577 - accuracy: 0.5987 - val_loss: 2.2377 - val_accuracy: 0.5000
Epoch 6/50
12/12 [==============================] - 1s 82ms/step - loss: 0.6208 - accuracy: 0.6311 - val_loss: 1.3194 - val_accuracy: 0.4500
Epoch 7/50
12/12 [==============================] - 1s 82ms/step - loss: 0.6426 - accuracy: 0.6276 - val_loss: 1.9199 - val_accuracy: 0.5250
Epoch 8/5

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

y_true = test_generator.classes
y_pred = model.predict(test_generator, steps = step_size_test)

In [6]:
y_pred = (y_pred > 0.5).astype(int).reshape(y_true.shape)

print(np.mean(y_true == y_pred))

pd.DataFrame(
  confusion_matrix(y_true, y_pred), 
  index=['true:yes', 'true:no'], 
  columns=['pred:yes', 'pred:no']
)

0.575


,pred:yes,pred:no
true:yes,10,10
true:no,7,13


In [7]:
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras import Model

base_model = MobileNet(weights='imagenet', include_top=False,
  input_shape = (224, 224, 3))

In [8]:
l = base_model.output
l = GlobalAveragePooling2D()(l)
out = Dense(1, activation='sigmoid')(l)

model = Model(inputs = base_model.input, outputs = out)

In [9]:
for layer in model.layers[:20]:
    layer.trainable = False
for layer in model.layers[20:]:
    layer.trainable = True

model.compile(loss='binary_crossentropy',
  optimizer='adam', metrics=['accuracy'])

In [10]:
train_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    rotation_range = 30,
    zoom_range = 0.15,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.15,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

valid_datagen = ImageDataGenerator(
  preprocessing_function = preprocess_input
)

test_datagen = ImageDataGenerator(
  preprocessing_function = preprocess_input
)


In [11]:
train_generator = train_datagen.flow_from_directory(
  'dogs/train/',
  target_size = (224,224),
  color_mode = 'rgb',
  batch_size = 10,
  class_mode = 'binary',
  shuffle = True,
  seed = 42
)

valid_generator = valid_datagen.flow_from_directory(
  'dogs/valid/',
  target_size = (224,224),
  color_mode = 'rgb',
  batch_size = 10,
  class_mode = 'binary',
  shuffle = False,
  seed = 42
)

test_generator = test_datagen.flow_from_directory(
  'dogs/test/',
  target_size = (224,224),
  color_mode = 'rgb',
  batch_size = 10,
  class_mode = 'binary',
  shuffle = False,
  seed = 42
)

step_size_train = train_generator.n // train_generator.batch_size
step_size_valid = valid_generator.n // valid_generator.batch_size
step_size_test = test_generator.n // test_generator.batch_size

callbacks = [EarlyStopping(monitor='val_loss', patience=5,
  restore_best_weights=True)]

Found 120 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [12]:
history = model.fit(train_generator, steps_per_epoch = step_size_train,
  epochs = 50, callbacks = callbacks, validation_data = valid_generator,
  validation_steps = step_size_valid, verbose = 1)

Epoch 1/50
12/12 [==============================] - 5s 160ms/step - loss: 0.7845 - accuracy: 0.6244 - val_loss: 1.0187 - val_accuracy: 0.7750
Epoch 2/50
12/12 [==============================] - 2s 124ms/step - loss: 0.4089 - accuracy: 0.8400 - val_loss: 1.1782 - val_accuracy: 0.7500
Epoch 3/50
12/12 [==============================] - 2s 124ms/step - loss: 0.2609 - accuracy: 0.9350 - val_loss: 9.8080 - val_accuracy: 0.5000
Epoch 4/50
12/12 [==============================] - 2s 123ms/step - loss: 0.2705 - accuracy: 0.9454 - val_loss: 14.2747 - val_accuracy: 0.5000
Epoch 5/50
12/12 [==============================] - 2s 123ms/step - loss: 0.1462 - accuracy: 0.9336 - val_loss: 5.2572 - val_accuracy: 0.5250
Epoch 6/50
12/12 [==============================] - 2s 123ms/step - loss: 0.3446 - accuracy: 0.9284 - val_loss: 6.6463 - val_accuracy: 0.5000


In [13]:
y_true = test_generator.classes
y_pred = model.predict(test_generator, steps = step_size_test)

In [14]:
# np.save('y_pred_dogs.npy', y_pred)

In [15]:
y_pred = (y_pred > 0.5).astype(int).reshape(y_true.shape)

print(np.mean(y_true == y_pred))

pd.DataFrame(
  confusion_matrix(y_true, y_pred), 
  index=['true:yes', 'true:no'], 
  columns=['pred:yes', 'pred:no']
)

0.725


,pred:yes,pred:no
true:yes,17,3
true:no,8,12
